По представленным в датасете данным проведите анализ результатов проведенного A/B-тестирования для сайта. Для этого:

1.	Проанализируйте: есть ли пользователи, которые попали в обе группы тестирования. Если да, то исключите их.
2.	Подумайте, как поступить с нулевыми значениями выручки. Нужно ли фильтровать данные? Или нулевые значения свидетельствуют об отсутствии изменений?
3.	Исключите дубли в записях для одного и того же пользователя.
4.	Проанализируйте выбросы в данных с помощью диаграммы размаха («ящика с усами»). Поработайте с ними, чтобы исключить вероятность их влияния на результаты тестирования.
5.	Проверьте распределение на нормальность, используя разные статистические тесты (например, Шапиро-Уилка). Если нужно, приведите данные к нормальному виду.
6.	Если удалось привести данные к нормальному виду при наименьшей потере данных, используйте параметрические тесты для А/В теста, предполагающие нормальность распределения. Если привести к нормальному виду не представляется возможным без большой потери данных, используйте непараметрические тесты (Манна-Уитни, Колмогорова-Смирнова и пр.), не предполагающие нормального распределения.
7.	Сделайте выводы по результатам A/B-тестирования. Визуализируйте распределение выручки для контрольной версии сайта (А) с его измененной версией (В).


### Обработка данных. Предварительный этап

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
df = pd.read_csv("AB_Test_Results.csv")

In [3]:
#выведите первые строки датасета
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [4]:
#выведите длину датасета
df.shape

(10000, 3)

In [5]:
#выведите описательные статистики по датасету
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [6]:
#удалите пользователей, которые есть в двух группах
#для этого через groupby посчитайте, сколько для каждого пользователя представлено групп (1 или 2)
user_group_counts = df.groupby('USER_ID')['VARIANT_NAME'].nunique()

#найдем индексы тех, у кого более 1 группы
users_in_both_groups = user_group_counts[user_group_counts > 1].index

In [7]:
#очищаем от них датасет
df_clean = df[~df['USER_ID'].isin(users_in_both_groups)]

In [8]:
#очищаем дубликаты по датасету
df_clean.drop_duplicates(subset=['USER_ID'], inplace=True)

C:\Users\tg082\AppData\Local\Temp\ipykernel_16812\2025887079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean.drop_duplicates(subset=['USER_ID'], inplace=True)


In [9]:
print(f"Количество пользователей, попавших в обе группы: {len(users_in_both_groups)}")
print(f"Размер датасета после удаления таких пользователей: {df_clean.shape}")
print(f"Размер датасета после удаления дубликатов: {df_clean.shape}")

Количество пользователей, попавших в обе группы: 119
Размер датасета после удаления таких пользователей: (9878, 3)
Размер датасета после удаления дубликатов: (9878, 3)


### Анализ выбросов

In [10]:
# Построим boxplot для анализа выбросов
plt.figure(figsize=(12, 6))
sns.boxplot(x='VARIANT_NAME', y='REVENUE', data=df_clean)
plt.title('Анализ выбросов по выручке')
plt.ylim(0, 20)  # Ограничиваем ось Y для лучшей визуализации
plt.show()

In [11]:
# Статистика по выручке до фильтрации выбросов
df_clean['REVENUE'].describe()

count    9878.000000
mean        0.095060
std         2.133522
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       196.010000
Name: REVENUE, dtype: float64

In [12]:
# Фильтрация выбросов с помощью метода IQR
Q1 = df_clean['REVENUE'].quantile(0.25)
Q3 = df_clean['REVENUE'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR

# Создаем копию для работы с выбросами
df_no_outliers = df_clean[df_clean['REVENUE'] <= upper_bound].copy()

In [13]:
print(f"Верхняя граница для фильтрации выбросов: {upper_bound}")
print(f"Количество удаленных выбросов: {len(df_clean) - len(df_no_outliers)}")
print(f"Новое количество записей: {len(df_no_outliers)}")

Верхняя граница для фильтрации выбросов: 0.0
Количество удаленных выбросов: 70
Новое количество записей: 9808


In [14]:
# Построим boxplot после фильтрации выбросов
plt.figure(figsize=(12, 6))
sns.boxplot(x='VARIANT_NAME', y='REVENUE', data=df_no_outliers)
plt.title('Анализ выбросов по выручке после фильтрации')
plt.ylim(0, 10)  # Ограничиваем ось Y для лучшей визуализации
plt.show()

### Проверка на нормальность распределения

In [15]:
# Разделим данные на контрольную и тестовую группы
control_group = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'control']['REVENUE']
variant_group = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'variant']['REVENUE']

In [16]:
# Тест Шапиро-Уилка для контрольной группы
stats.shapiro(control_group)

ShapiroResult(statistic=0.6363656520843506, pvalue=0.0)

In [17]:
# Тест Шапиро-Уилка для тестовой группы
stats.shapiro(variant_group)

ShapiroResult(statistic=0.6305386424064636, pvalue=0.0)

In [18]:
# Поскольку данные сильно не нормальны (p-value < 0.05), попробуем логарифмическое преобразование
# Добавим небольшое значение к выручке, чтобы избежать log(0)
df_no_outliers['LOG_REVENUE'] = np.log1p(df_no_outliers['REVENUE'])

In [19]:
# Разделим данные с логарифмированной выручкой
control_log = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'control']['LOG_REVENUE']
variant_log = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'variant']['LOG_REVENUE']

# Тест Шапиро-Уилка для логарифмированной выручки
print("Тест Шапиро-Уилка для логарифмированной выручки в контрольной группе:")
print(stats.shapiro(control_log))
print("\nТест Шапиро-Уилка для логарифмированной выручки в тестовой группе:")
print(stats.shapiro(variant_log))

Тест Шапиро-Уилка для логарифмированной выручки в контрольной группе:
statistic=0.8480190634727478, pvalue=0.0

Тест Шапиро-Уилка для логарифмированной выручки в тестовой группе:
statistic=0.8443257808685303, pvalue=0.0


In [20]:
# Визуализируем распределения
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(data=df_no_outliers, x='REVENUE', hue='VARIANT_NAME', bins=50, kde=True)
plt.title('Исходное распределение выручки')
plt.xlim(0, 10)

plt.subplot(1, 2, 2)
sns.histplot(data=df_no_outliers, x='LOG_REVENUE', hue='VARIANT_NAME', bins=50, kde=True)
plt.title('Логарифмированное распределение выручки')

plt.tight_layout()
plt.show()

### Статистический тест для A/B тестирования

In [21]:
# Поскольку данные не нормальны даже после преобразования, используем непараметрический тест Манна-Уитни
u_stat, p_value = stats.mannwhitneyu(control_group, variant_group)
print(f"Mann-Whitney U test: U-statistic={u_stat}, p-value={p_value}")

Mann-Whitney U test: U-statistic=1730368.0, p-value=0.11717730265361126


In [22]:
# Описательная статистика для обеих групп
control_stats = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'control']['REVENUE']
variant_stats = df_no_outliers[df_no_outliers['VARIANT_NAME'] == 'variant']['REVENUE']

print(f"Количество пользователей в контрольной группе: {len(control_stats)}")
print(f"Количество пользователей в тестовой группе: {len(variant_stats)}")
print(f"\nСредняя выручка в контрольной группе: {control_stats.mean():.4f}")
print(f"Средняя выручка в тестовой группе: {variant_stats.mean():.4f}")
print(f"\nМедианная выручка в контрольной группе: {control_stats.median():.4f}")
print(f"Медианная выручка в тестовой группе: {variant_stats.median():.4f}")

Количество пользователей в контрольной группе: 4895
Количество пользователей в тестовой группе: 4913

Средняя выручка в контрольной группе: 0.0968
Средняя выручка в тестовой группе: 0.0933

Медианная выручка в контрольной группе: 0.0000
Медианная выручка в тестовой группе: 0.0000


### Визуализация результатов

In [23]:
# Визуализация распределения выручки
plt.figure(figsize=(12, 6))

# Основное распределение с обрезанием высоких значений
sns.histplot(data=df_no_outliers, x='REVENUE', hue='VARIANT_NAME', bins=50, kde=True, alpha=0.5)
plt.title('Распределение выручки по группам (основная масса данных)')
plt.xlim(0, 10)
plt.xlabel('Выручка')
plt.ylabel('Количество пользователей')

plt.tight_layout()
plt.show()

In [24]:
# Сравнение средней выручки между группами
group_means = df_no_outliers.groupby('VARIANT_NAME')['REVENUE'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='VARIANT_NAME', y='REVENUE', data=group_means)
plt.title('Средняя выручка по группам')
plt.xlabel('Группа')
plt.ylabel('Средняя выручка')

# Добавим значение средней выручки на график
for i, row in group_means.iterrows():
    plt.text(i, row['REVENUE'] + 0.01, f"{row['REVENUE']:.4f}", 
             ha='center', va='bottom', fontsize=12)

plt.ylim(0, max(group_means['REVENUE']) * 1.2)
plt.show()